In [1]:
from tensorflow import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Dense, concatenate, GlobalAveragePooling2D, AveragePooling2D, Flatten, Dropout, Input
from keras.datasets import cifar10
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
import cv2
import numpy as np
import math

In [2]:
def load_cifar10_data(img_rows, img_cols):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train[0:1000, :, :]
    y_train = y_train[:1000]

    x_test = x_test[:500,:,:,:]
    y_test = y_test[:500]

    x_train = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_train])
    x_test = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_test])
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train = x_train / 255.0
    x_test = x_test / 255.0

    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = load_cifar10_data(224,224)
img_input = Input(shape=(224,224,3))

In [3]:
def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj, name=None, kernel_init='glorot_uniform', bias_init='zero'):
    conv_1x1 = Conv2D(filters_1x1, (1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3_reduce = Conv2D(filters_3x3_reduce,(1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3,3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3_reduce)
    conv_5x5_reduce = Conv2D(filters_5x5_reduce, (1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5,5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    max_pool = MaxPool2D((3,3), strides=(1,1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1,1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(max_pool)
    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    return output

In [4]:
def decay(epoch, steps=100): # epoch 마다 lerning rate 감소
    initial_lrate = 0.01
    drop = 0.96
    epoch_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1 +epoch)/ epoch_drop))
    return lrate

In [5]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)
input_layer = Input(shape=(224,224,3))

In [9]:
# 시작 부분
x = Conv2D(64, (7,7), padding='same', strides=(2,2), activation='relu', 
           name='conv_1_7x7-2', kernel_initializer = kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3,3), strides=(2,2), name='max_pool_1_3x3-2', padding='same')(x)

x = Conv2D(192, (3,3), padding='same', strides=(1,1), activation='relu',
           name='conv_2_3x3-1', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
x = MaxPool2D((3,3), strides=(2,2), name='max_pool_2_3x3-2', padding='same')(x)

# 중간 부분
x = inception_module(x, 64, 96, 128, 16, 32, 32, name='inception_3a', kernel_init=kernel_init,bias_init=bias_init)
x = inception_module(x, 128, 128, 192, 32, 96, 64, name='inception_3b', kernel_init=kernel_init, bias_init=bias_init)
x = MaxPool2D((3,3), strides=(2,2), name='max_pool_3_3x3-2')(x)
x = inception_module(x, 192, 96, 208, 16, 48, 64, name='inception_4a')

#중간 부분의 첫 번쨰 보조 분류기
x1 = AveragePooling2D((5,5), strides=3, name='avg_pool_aux_1')(x)
x1 = Conv2D(128, (1,1), padding='same', activation='relu', name='conv_aux_1')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu', name='dense_aux_1')(x1)
x1 = Dense(10, activation='softmax', name='aux_output_1')(x1)

x = inception_module(x, 160, 112, 224, 24, 64, 64, name='inception_4b', kernel_init=kernel_init, bias_init=bias_init)
x = inception_module(x, 128, 128, 256, 24, 64, 64, name='inception_4c', kernel_init=kernel_init, bias_init=bias_init)
x = inception_module(x, 112, 144, 288, 32, 64, 64, name='inception_4d', kernel_init=kernel_init, bias_init=bias_init)

# 중간 부분의 두 번쨰 보조 분류지
x2 = AveragePooling2D((5, 5), strides=3, name='avg_pool_aux_2')(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu', name='conv_aux_2')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu', name='dense_aux_2')(x2)
x2 = Dense(10, activation='softmax', name='aux_output_2')(x2)

x = inception_module(x, 256, 160, 320, 32, 128, 128, name='inception_4e', kernel_init=kernel_init, bias_init=bias_init)
x = MaxPool2D((3,3), strides=(2,2), name='max_pool_4_3x3-2')(x)
x = inception_module(x, 256, 160, 320, 32, 128, 128, name='inception_5a', kernel_init=kernel_init, bias_init=bias_init)
x = inception_module(x, 384, 192, 384, 48, 128, 128, name='inception_5b', kernel_init=kernel_init, bias_init=bias_init)

# 끝 부분
x = GlobalAveragePooling2D(name='global_avg_pool_5_3x3-1')(x)
x = Dropout(0.4)(x)
x = Dense(10, activation='softmax', name='output')(x)

# 모델 저장
model = Model(input_layer, [x ,x1, x2], name='GoogLeNet')

In [14]:
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=False)
lr_sc = LearningRateScheduler(decay, verbose=1)
model.compile(loss=['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'],
              loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=[['accuracy'], ['accuracy'], ['accuracy']])
model.fit(x_train, [y_train, y_train, y_train], validation_data=(x_test, [y_test, y_test, y_test]), epochs=3, callbacks=[lr_sc])


Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - aux_output_1_accuracy: 0.1068 - aux_output_1_loss: 0.6996 - aux_output_2_accuracy: 0.0929 - aux_output_2_loss: 0.7059 - loss: 3.9763 - output_accuracy: 0.1157 - output_loss: 2.5707 - val_aux_output_1_accuracy: 0.0980 - val_aux_output_1_loss: 0.6925 - val_aux_output_2_accuracy: 0.1080 - val_aux_output_2_loss: 0.6955 - val_loss: 3.7658 - val_output_accuracy: 0.1080 - val_output_loss: 2.3798 - learning_rate: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.01.
Epoch 2/3
32/32 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - aux_output_1_accuracy: 0.1284 - aux_output_1_loss: 0.6909 - aux_output_2_accuracy: 0.0934 - aux_output_2_loss: 0.6929 - loss: 3.7699 - output_accuracy: 0.0861 - output_loss: 2.3862 - val_aux_output_1_accuracy: 0.0820 - val_aux_output_1_loss: 0.6905 - val_aux_output_2_accuracy: 0.0820 - val_aux_output_2_loss: 0.6921 - val_loss: 3.7603 - val_output_accurac

In [15]:
score = model.evaluate(x_test, y_test)
print('테스트 정확도 :', score[1])

16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 522ms/step - aux_output_1_accuracy: 0.0000e+00 - aux_output_1_loss: 0.0000e+00 - aux_output_2_accuracy: 0.0000e+00 - aux_output_2_loss: 0.0000e+00 - loss: 2.3025 - output_accuracy: 0.1509 - output_loss: 2.3025
테스트 정확도 : 2.3200290203094482
